# Code for scraping data off of the H1B Visa website

https://github.com/yiuhyuk/ds_salary_h1b/blob/master/h1b_salary.ipynb

https://towardsdatascience.com/how-much-do-data-scientists-make-cbd7ec2b458

In [1]:
link = 'https://h1bdata.info/index.php?em=&job=&city='
year = '&year=All+Years'

Use other code to download names of cities with H1B visas

In [2]:
import pandas
cities = pandas.read_csv('cities.csv')

Format city names into browser friendly format

In [3]:
cities = ['+'.join(str(i).split()) for i in cities.Cities]

In [4]:
import requests
from bs4 import BeautifulSoup

Download all the data from the H1B website

In [5]:
jobs = []
for i in cities:
    page = link+i+year # create web address
    response = requests.get(page) #open link # ,timeout=1000
    content = BeautifulSoup(response.content,'lxml') # get info
    
    for row in content.find_all('tr')[1:]:
        # find all tables on webpage
        # for each row in the table found exclude header
        row_data = []
        for i in row:
            # for each column in a row
            row_data.append(i.text)
            # append data without html, only readable text
        jobs.append(row_data)
        #create table of all job info

In [6]:
import webbrowser
url = 'https://www.youtube.com/watch?v=vVCT4iGzm8Q&list=WL&index=84&t=0s'
webbrowser.open(url, new=1, autoraise=True)

True

In [7]:
header = ['company','title','salary','location','date']

In [8]:
import pandas

Convert lists of jobs info into a dataframe

In [9]:
jobsDF = pandas.DataFrame()
column = 0
for i in header:
    jobsDF[i] = [j[column] for j in jobs]
    column += 1
jobsDF.head()

,company,title,salary,location,date
0,CANACCORD GENUITY INC,(2ND YEAR) ANALYST,"95,160","NEW YORK, NY",08/02/2016
1,COMMONWEALTH BANK OF AUSTRALIA,"(BCF) ASSOCIATE DIRECTOR, SAF EXECUTION (NEW Y...","125,000","NEW YORK, NY",01/08/2016
2,COMMONWEALTH BANK OF AUSTRALIA,"(BCF) ASSOCIATE DIRECTOR, SAF EXECUTION (NEW Y...","130,047","NEW YORK, NY",10/13/2017
3,TWO SIGMA INVESTMENTS LP,(COMPUTER SYSTEMS) RESEARCH ANALYST,"150,000","NEW YORK, NY",03/14/2016
4,TWO SIGMA INVESTMENTS LP,(COMPUTER SYSTEMS) RESEARCHER,"160,000","NEW YORK, NY",01/19/2017


Format Columns properly by converting strings to numbers, dates or years

In [10]:
jobsDF['salary'] = [i.replace(',','') for i in jobsDF.salary]
jobsDF['salary'] = jobsDF['salary'].astype(float)
jobsDF['date'] = pandas.to_datetime(jobsDF['date'])
jobsDF['year'] = [i.year for i in jobsDF['date']]

In [11]:
jobsDF.head()

,company,title,salary,location,date,year
0,CANACCORD GENUITY INC,(2ND YEAR) ANALYST,95160.0,"NEW YORK, NY",2016-08-02,2016
1,COMMONWEALTH BANK OF AUSTRALIA,"(BCF) ASSOCIATE DIRECTOR, SAF EXECUTION (NEW Y...",125000.0,"NEW YORK, NY",2016-01-08,2016
2,COMMONWEALTH BANK OF AUSTRALIA,"(BCF) ASSOCIATE DIRECTOR, SAF EXECUTION (NEW Y...",130047.0,"NEW YORK, NY",2017-10-13,2017
3,TWO SIGMA INVESTMENTS LP,(COMPUTER SYSTEMS) RESEARCH ANALYST,150000.0,"NEW YORK, NY",2016-03-14,2016
4,TWO SIGMA INVESTMENTS LP,(COMPUTER SYSTEMS) RESEARCHER,160000.0,"NEW YORK, NY",2017-01-19,2017


In [12]:
# Sort by company and year
#jobsDF.sort_values(by=['company','year'], inplace=True, ascending=True)

In [13]:
len(jobsDF)

2575533

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10,6))
ax = sns.barplot(x=jobsDF['year'].value_counts().sort_index().index, 
                 y=jobsDF['year'].value_counts().sort_index().values)
ax.set_xlabel("Year",fontsize=16)
ax.set_ylabel("Number of H1B Visas",fontsize=16)
plt.tight_layout()
plt.savefig(fname='num_jobs', dpi=150)

In [ ]:
jobsDF.salary.median()

In [ ]:
jobsDF.columns

In [14]:
len(jobsDF[jobsDF.location == 'NEW YORK, NY'])

99934

In [15]:
jobsDF.to_csv('H1B_Visas.csv', sep=",", index=False)